In [1]:
pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np

In [2]:
print(cv2.__version__)

4.8.0


In [10]:
# # 加载YOLO模型
# net = cv2.dnn.readNet(r"C:\Users\gdnjr5233_YOLO\Desktop\YOLOv3\yolov3.weights", r"C:\Users\gdnjr5233_YOLO\Desktop\YOLOv3\yolov3.cfg")

# # 获取输出层的名称
# layer_names = net.getUnconnectedOutLayersNames()

# # 加载类别标签文件
# with open(r"C:\Users\gdnjr5233_YOLO\Desktop\YOLOv3\traffic_sign.txt", "r") as f:
#     classes = [line.strip() for line in f.readlines()]

# # 加载图像
# image = cv2.imread(r"C:\Users\gdnjr5233_YOLO\Desktop\YOLOv3\test_2.jpg")

# # 获取图像的高度和宽度
# height, width, _ = image.shape

# # 创建一个blob（二进制大对象）从图像中获取信息
# blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

# # 将blob输入到模型中
# net.setInput(blob)

# # 运行前向传播
# outputs = net.forward(layer_names)

# # 遍历输出层并提取检测结果
# for output in outputs:
#     for detection in output:
#         scores = detection[5:]
#         class_id = np.argmax(scores)
#         confidence = scores[class_id]

#         if confidence > 0.5:
#             # 获取类别名称
#             class_name = classes[class_id]
#             # 检测框的坐标
#             center_x = int(detection[0] * width)
#             center_y = int(detection[1] * height)
#             w = int(detection[2] * width)
#             h = int(detection[3] * height)

#             # 检测框的左上角坐标
#             x = int(center_x - w / 2)
#             y = int(center_y - h / 2)

#             # 在图像上绘制检测框
#             cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# # 显示结果
# cv2.imshow("Result", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

# 加载YOLO模型
net = cv2.dnn.readNet(r"C:\Users\gdnjr5233_YOLO\Desktop\YOLOv3\yolov4.cfg", r"C:\Users\gdnjr5233_YOLO\Desktop\YOLOv3\yolov4.weights")

# 获取输出层的名称
layer_names = net.getUnconnectedOutLayersNames()

# 加载类别标签文件
with open(r"C:\Users\gdnjr5233_YOLO\Desktop\YOLOv3\traffic_sign_coco_names.txt") as f:
    classes = [line.strip() for line in f.readlines()]

# 加载图像
image = cv2.imread(r"C:\Users\gdnjr5233_YOLO\Desktop\YOLOv3\test_6_stop.jpg")

# 获取图像的高度和宽度
height, width, _ = image.shape

# 创建一个blob（二进制大对象）从图像中获取信息
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

# 将blob输入到模型中
net.setInput(blob)

# 运行前向传播
outputs = net.forward(layer_names)

# 初始化列表，用于存储框坐标、置信度和类别
bboxes = []
confidences = []
class_ids = []

# 遍历输出层并提取检测结果
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5:
            # 检测框的坐标
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # 检测框的左上角坐标
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # 保存框的信息
            bboxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# 非极大值抑制
confidence_threshold = 0.5  # 置信度阈值
nms_threshold = 0.4  # NMS阈值
indices = cv2.dnn.NMSBoxes(bboxes, confidences, confidence_threshold, nms_threshold)

# 遍历保留的框索引并在图像上绘制检测框
for i in indices.flatten():
    x, y, w, h = bboxes[i]
    class_id = class_ids[i]

    # 跳过不想显示的标签
    # if classes[class_id] in ['speed_limit_30', 'speed_limit_40', 'speed_limit_50', 'traffic light']:
    #     continue

    # 在图像上绘制检测框
    color = (0, 255, 0)  # 绿色
    cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)

    # 设置字体参数
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5  # 增加字体大小
    font_thickness = 2  # 增加字体粗细

    # 设置文字颜色
    text_color = (0, 255, 0)  # 绿色

    # 在检测框上显示类别名称和置信度
    text = f"{classes[class_id]}: {confidences[i]:.2f}"
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    text_x = x
    text_y = y - 10  # 将文字上移一些，以不与检测框紧密连接

    # 绘制文字轮廓
    cv2.putText(image, text, (text_x, text_y), font, font_scale, (0, 0, 0), font_thickness + 1, cv2.LINE_AA)

    # 绘制文字
    cv2.putText(image, text, (text_x, text_y), font, font_scale, text_color, font_thickness, cv2.LINE_AA)


    # # 在图像上绘制检测框
    # color = (0, 255, 0)  # 绿色
    # cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)

    # # 在检测框上显示类别名称和置信度
    # text = f"{classes[class_id]}: {confidences[i]:.2f}"
    # cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# 显示结果
cv2.imshow("Result", image)
cv2.waitKey(0)
cv2.destroyAllWindows()